In [14]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

2022-07-07 00:40:06,586 INFO NumExpr defaulting to 4 threads.


In [2]:
# Set Up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


2022-07-06 14:31:26,381 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-06 14:31:26,442 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-06 14:31:26,453 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip


2022-07-06 14:31:27,107 INFO About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\akemu\.wdm\drivers\chromedriver\win32\103.0.5060.53]


2022-07-06 14:31:30,073 INFO Driver has been saved in cache [C:\Users\akemu\.wdm\drivers\chromedriver\win32\103.0.5060.53]


In [3]:
# assign the url and instruct the browser to visit:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# set up the HTML parser and convert to a soup object::
html = browser.html
news_soup = soup(html, 'html.parser')
# assigned slide_elem as the variable to look for the <div /> tag and its descendent ( parent element):
slide_elem = news_soup.select_one('div.list_text')

In [6]:
# scrape the article’s title
# assign the title and summary text to variables:
slide_elem.find('div', class_='content_title')

<div class="content_title">3 Things We've Learned From NASA's Mars InSight </div>

In [7]:
# get just the text, and the extra HTML stuff isn't necessary:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

# (With this new code, we’re searching within that element for the title. We’re also stripping the additional HTML attributes and tags with the use of .get_text().)

"3 Things We've Learned From NASA's Mars InSight "

In [8]:
# scrape the article summary instead of the title: 
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Scientists are finding new mysteries since the geophysics mission landed two years ago.'

# Scrape Mars Data: Featured Image:
### Featured Images

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# there are only three buttons;
# we want to click the full-size image button, 
# we can go ahead and use the HTML tag in our code:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [11]:
# parse and scrape the full-size image URL:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# use the image tag and class (<img />and fancybox-img) to build the URL to the full-size image:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

# tell BeautifulSoup to look inside the <img /> tag for an image with a class of fancybox-image

'image/featured/mars1.jpg'

In [13]:
# add the base URL to our code:
# This variable holds our f-string (img_url);
# This is an f-string, a type of string formatting used for print statements in Python (f'https://spaceimages-mars.com/');
# The curly brackets hold a variable that will be inserted into the f-string when it's executed({img_url_rel})
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [15]:
#(#creating a new DataFrame from the HTML table)
# read_html() specifically searches for and returns a list of tables found in the HTML. 
# By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, or the first item in the list. 
# Then, it turns the table into a DataFrame.
df = pd.read_html('https://galaxyfacts-mars.com')[0] 
# assign columns to the new DataFrame for additional clarity.
df.columns=['description', 'Mars', 'Earth']
# using the .set_index() function, we're turning the Description column into the DataFrame's index. 
# inplace=True means that the updated index will remain in place, without having to reassign the DataFrame to a new variable.
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [21]:
# convert our DataFrame back into HTML-ready code using .to_html()
df.to_html()

# result is a slightly confusing-looking set of HTML code—it's a <table /> element with a lot of nested elements.

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [22]:
# execute that cell to end the session.
browser.quit()



In [23]:
# 10.4.1 Store the Data
# db.collectionName.insertOne({key:value}). Its components do the following:

# db refers to the active database, practicedb.
# collectionName is the name of the new collection we're creating (we'll customize it when we practice).
# .insertOne({ }) is how MongoDB knows we're inserting data into the collection.
# key:value is the format into which we're inserting our data; its construction is very similar to a Python dictionary.
# for example: db.zoo.insertOne({name: 'Cleo', species: 'jaguar', age: 12, hobbies: ['sleeping', 'eating', 'climbing']})
# type "show collections" we'll actually see a result: zoo.
# view what's inside a collection with the find() command.
# Documents can also be deleted or dropped. The syntax to do so follows: db.collectionName.deleteOne({}).
# empty the collection at once:  db.zoo.remove({})
# remove a collection all together, we would use db.zoo.drop().
# to remove the test database, we will use this line of code: db.dropDatabase().
# quit the Mongo shell by using keyboard commands: CTRL + C for Windows. 